# Analysis of results from simulated data
Deeper look into a model's performance on simulated data.

In [9]:
# Imports
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from master_scripts.data_functions import (load_experiment, get_git_root, separation_distance, energy_difference,
                                           relative_energy, event_indices, normalize_image_data)
from master_scripts.analysis_functions import (doubles_classification_stats, singles_classification_stats)
from sklearn.metrics import f1_score
%load_ext autoreload
%autoreload 2
repo_root = get_git_root()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data and experiment import
Load the image data and split into training and validation sets. Since we specify the random seed, we can
reproduce the exact same data the model was originally validated on to explore it.

In [2]:
images = np.load(repo_root + "data/simulated/images_full_pixelmod.npy")
positions = np.load(repo_root + "data/simulated/positions_full.npy")
energies = np.load(repo_root + "data/simulated/energies_full.npy")
labels = np.load(repo_root + "data/simulated/labels_full.npy")

In [5]:
# Load experiment and associated model (must be a saved model instance complete with weights)
#experiment_id = "40350020681b"
experiment_id = "9886d077068a"
experiment = load_experiment(experiment_id)
model = tf.keras.models.load_model(repo_root + "models/" + experiment_id + ".h5")
# Print experiment metrics
print("==== Experiment metrics")
print(json.dumps(experiment["metrics"], indent=2))
print("====")

==== Experiment metrics
{
  "accuracy_score": 0.9832926315789474,
  "confusion_matrix": {
    "TN": 236349,
    "FP": 1183,
    "FN": 6753,
    "TP": 230715
  },
  "f1_score": 0.9830920859201562,
  "matthews_corrcoef": 0.9668510918102141,
  "roc_auc_score": 0.9928096454052002
}
====


## Predict on validation data

In [6]:
x_idx = np.arange(len(images))
train_idx, val_idx = train_test_split(
    x_idx,
    random_state=experiment['experiment_config']['random_seed']
)
# Predict on the validation set
prediction = model.predict(normalize_image_data(images[val_idx]))
val_pred = (prediction > 0.5).astype(int)

### All, close, all without close statistics

In [19]:
s_idx, d_idx, c_idx = event_indices(positions[val_idx])
non_close_idx = np.setdiff1d(np.concatenate((s_idx, d_idx), axis=0), c_idx)
f1_close = f1_score(labels[val_idx][c_idx], val_pred[c_idx])
f1_non_close = f1_score(labels[val_idx][non_close_idx], val_pred[non_close_idx])
print(f1_close)
print(f1_non_close)

test = {}
test['val_idx'] = val_idx.tolist()
test['s_idx'] = s_idx.tolist()
test['d_idx'] = d_idx.tolist()
test['c_idx'] = c_idx.tolist()

with open("idx_array_storagetest.json", 'w') as fp:
    json.dump(test, fp)

0.6121237812632472
0.9868588695098226


# Single events
## Descriptive stats on validation data

In [7]:
singles = singles_classification_stats(positions[val_idx], energies[val_idx], val_pred)

In [8]:
# group by classification
sstats = singles[singles.columns[:-1]].groupby('classification')
# Aggregate the desired statistics into a new df
sstats = sstats.agg([np.mean, np.std, np.median]).applymap('{:.3f}'.format)
print(json.dumps(experiment['metrics']['confusion_matrix'], indent=2))
display(sstats)

{
  "TN": 236349,
  "FP": 1183,
  "FN": 6753,
  "TP": 230715
}


x_pos                y_pos               energy              
                 mean    std median   mean    std median   mean    std median
classification                                                               
0               7.991  4.233  7.993  8.003  4.232  8.004  0.499  0.289  0.498
1               8.428  4.310  8.654  7.430  4.023  7.303  0.742  0.199  0.779

## Plot some desired number of misclassified single event

In [ ]:
n_events = 16
s_mc = singles.loc[singles['classification'] == 1]
selected = np.random.choice(s_mc['indices'].values, n_events, replace=False)
fig, ax = plt.subplots(np.ceil(n_events/4).astype(np.int32), 4, figsize=(16, 16))
#fig.set_facecolor('grey')
for i, idx in enumerate(selected):
    a = ax.flatten()[i]
    event = s_mc.loc[s_mc['indices'] == idx]
    x = event['x_pos'].values[0].round(2)
    y = event['y_pos'].values[0].round(2)
    energy = event['energy'].values[0].round(2)
    sns.heatmap(images[val_idx][idx].reshape(16, 16), square=True, ax=a)
    #a.text(0, 15 + 0.6, f"",
    #    fontsize=8,
    #    color='white'
    #    )
    a.set_title(f"{idx}, ({x}, {y}), E={energy}", fontsize=10)
    a.invert_yaxis()

In [ ]:
# Check positions of all misclassified single events
print(np.count_nonzero(positions[val_idx][s_mc['indices'].values, 2] != -100))

## Distribution of positions and energies
Where are the misclassified singles located, and what are their energies?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
#fig.set_facecolor('grey')
sns.distplot(s_mc['energy'], kde=False, ax=ax[0])
sns.scatterplot(s_mc['x_pos'], s_mc['y_pos'], alpha=0.5, ax=ax[1])
ax[0].set_title("Energy distribution for singles classified as double")
ax[1].set_title("Locations of misclassified single events")

# Double events
## Descriptive statistics on validation data

In [ ]:
# Load dataframes
doubles = doubles_classification_stats(positions[val_idx], energies[val_idx], val_pred, scale=True)

In [ ]:
# Group by close events, then by which class the event was classified as
dstats = doubles[doubles.columns[:-1]].groupby(['close', 'classification'])
# Aggregate the desired statistics into a new df
dstats = dstats.agg([np.mean, np.std, np.median, np.min, np.max]).applymap('{:.3f}'.format)
print(json.dumps(experiment['metrics']['confusion_matrix'], indent=2))
display(dstats)

## Distributions and scatterplot

In [ ]:
dgroup = doubles.groupby(['close', 'classification'])

### Comparing correct and misclassified double events
#### Separation distances

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
keys = list(dgroup.groups.keys())
sns.distplot(dgroup.get_group(keys[0])['separation distance'], kde=False, label=keys[0][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[1])['separation distance'], kde=False, label=keys[1][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[2])['separation distance'], kde=False, label=keys[2][1], ax=ax[1])
sns.distplot(dgroup.get_group(keys[3])['separation distance'], kde=False, label=keys[3][1], ax=ax[1])
ax[0].set_title("Separation distances for non-close events")
ax[0].legend()
ax[1].set_title("Separation distances for close events")
ax[1].legend()

#### Relative energy

In [ ]:
dgroup = doubles.groupby(['close', 'classification'])
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
keys = list(dgroup.groups.keys())
#bins = 
sns.distplot(dgroup.get_group(keys[0])['relative energy'], kde=False, label=keys[0][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[1])['relative energy'], kde=False, label=keys[1][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[2])['relative energy'], kde=False, label=keys[2][1], ax=ax[1])
sns.distplot(dgroup.get_group(keys[3])['relative energy'], kde=False, label=keys[3][1], ax=ax[1])
ax[0].set_title("Relative energies for non-close events")
ax[0].legend()
ax[1].set_title("Relative energies for close events")
ax[1].legend()

#### Energy difference

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))
keys = list(dgroup.groups.keys())
sns.distplot(dgroup.get_group(keys[0])['energy difference'], kde=False, label=keys[0][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[1])['energy difference'], kde=False, label=keys[1][1], ax=ax[0])
sns.distplot(dgroup.get_group(keys[2])['energy difference'], kde=False, label=keys[2][1], ax=ax[1])
sns.distplot(dgroup.get_group(keys[3])['energy difference'], kde=False, label=keys[3][1], ax=ax[1])
ax[0].set_title("Energy difference for non-close events")
ax[0].legend()
ax[1].set_title("Energy difference for close events")
ax[1].legend()

#### ROC

In [ ]:
dist_bins = np.arange(0, np.amax(rel_distance_test), 0.5)
energy_bins = np.arange(0, np.amax(energy_diff_test), 0.02)
fig, ax = plt.subplots(1, 2, figsize=(12,4))
ax[0].hist(rel_distance_test[double_indices][correct_doubles], bins=dist_bins, alpha=0.5, label="correct")
ax[0].hist(rel_distance_test[double_indices][wrong_doubles], bins=dist_bins, alpha=0.5, label="wrong")
ax[0].set_title("Distribution of separation distances\n for classified double events")
ax[0].set_xlabel("Separation distance [mm]")
ax[0].set_ylabel("Number of events")
ax[0].legend()
ax[1].hist(rel_energy_test[double_indices][correct_doubles], bins=energy_bins, alpha=0.5, label="correct")
ax[1].hist(rel_energy_test[double_indices][wrong_doubles], bins=energy_bins, alpha=0.5, label="wrong")
ax[1].set_title("Distribution of relative energy \n for classified double events")
ax[1].set_xlabel("Relative energy")
ax[1].set_ylabel("Number of events")
ax[1].legend()
fig.savefig(FIGURE_PATH+net+"_relative_test_compare.pdf", format="pdf")

In [ ]:
dist_bins = np.arange(0, np.amax(rel_distance_test), 0.5)
energy_bins = np.arange(0, 10, 0.1)
fig, ax = plt.subplots(1, 2, figsize=(12,4))
#ax[0].hist(rel_distance_test[double_indices][correct_doubles], bins=dist_bins, alpha=0.5, label="correct")
ax[0].hist(rel_distance_test[double_indices][wrong_doubles], bins=dist_bins, alpha=0.5, label="wrong")
ax[0].set_title("Distribution of Relative distances\n for classified double events")
ax[0].set_xlabel("Relative distance [mm]")
ax[0].set_ylabel("Number of events")
ax[0].legend()
#ax[1].hist(rel_energy_test[double_indices][correct_doubles], bins=energy_bins, alpha=0.5, label="correct")
#ax[1].hist(rel_energy_test[double_indices][wrong_doubles], bins=energy_bins, alpha=0.5, label="wrong")
ax[1].hist(rel_energy_test[double_indices][wrong_doubles], label="wrong")
ax[1].set_title("Distribution of relative energy \n for classified double events")
ax[1].set_xlabel("Relative energy")
ax[1].set_ylabel("Number of events")
ax[1].legend()
fig.savefig(FIGURE_PATH+net+"_relative_test_compare.pdf", format="pdf")

### Scatterplot relative distance vs. relative energy

In [ ]:
plt.scatter(
    rel_distance_test[double_indices][wrong_doubles], 
    rel_energy_test[double_indices][wrong_doubles],
    marker='.',
    )
plt.title("Separation distance vs. relative energy for misclassified double events")
plt.xlabel("Separation distance [mm]")
plt.ylabel("Relative energy")
plt.show()

# Distribution of position around highest intensity pixel
In previous work data analysis showed that most event positions are within the highest intensity pixel,
and all (verify!) events are within the two highest intensity pixels,
It might be reasonable to look at how the predicted positions are distributed around the highest intensity
pixel.

In [ ]:
# Reshape input array to a 2D array with rows being kept as with original array.
# Then, get idnices of max values along the columns.
pix_hmap = np.zeros(images[0].shape)
max_idx = images.reshape(images.shape[0],-1).argmax(1)
# Get unravel indices corresponding to original shape of A
maxpos_vect = np.column_stack(np.unravel_index(max_idx, images[0,:,:].shape))
np.add.at(
    pix_hmap, 
    (
        maxpos_vect[:, 0],
        maxpos_vect[:, 1],
        maxpos_vect[:, 2]
    ),
    1
)
#pix_hmap[maxpos_vect[:, 0], maxpos_vect[:, 1], maxpos_vect[:, 2]] += 1
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title('Heatmap of highest intensity pixels in dataset')
sns.heatmap(pix_hmap.reshape((16,16)), ax=ax, square=True)

In [ ]:
imgs = images[single_indices].reshape(images[single_indices].shape[0],16,16)

# get index of highest energy pixel
print(np.unravel_index(np.argmax(imgs[0], axis=None), imgs[0].shape))
fix, ax = plt.subplots()
ax.imshow(imgs[0])
ax.plot(0,0, 'rx')

In [ ]:
config = {
    "DATA_PATH": "../../data/real/anodedata.txt",              
    "MODEL_PATH": "../../data/output/models/",                
    "CLASSIFIER": "Project-0.97.hdf5",                      
    "SINGLE_ENERGY_MODEL": "single_energy_model_name.hdf5",    
    "SINGLE_POSITION_MODEL": "single_position_model_name.hdf5",
    "DOUBLE_ENERGY_MODEL": "double_energy_model_name.hdf5",    
    "DOUBLE_POSITION_MODEL": "double_position_model_name.hdf5" 
}

data = import_real_data(config)
print(data['image'].type)
